In [15]:
import json

In [68]:
import json
pwc_df_1 = [json.loads(line) for line in open('/net/nfs.corp/s2-research/beltagy/result_extraction/data/pwc_s2.jsonl')]
pwc_df = [json.loads(line) for line in open('data/pwc_s2.jsonl')]

import pandas as pd
pwc_df = pd.DataFrame(pwc_df)
pwc_df_1 = pd.DataFrame(pwc_df_1)
pwc_df['dataset'] = pwc_df_1['dataset']

In [43]:
pwc_sentences = [json.loads(line) for line in open('data/pwc_sentences.jsonl')]

In [49]:
from tqdm import tqdm
pwc_output = []
for line in tqdm(open('outputs/pwc_sentences_predict_total.jsonl')) :
    line = json.loads(line)
    del line['logits']
    del line['mask']
    pwc_output.append(line)

363170it [00:53, 6851.44it/s]


In [58]:
for s, t in zip(pwc_sentences, pwc_output) :
    s.update(t)

In [60]:
pwc_sentences = pd.DataFrame(pwc_sentences)
del pwc_output

In [71]:
pwc_sentences = pwc_sentences.groupby(['doc_id', 'text_field'], as_index=False).aggregate(lambda x : tuple(x))

In [100]:
pwc_df.columns

Index(['category', 'metric', 'model_name', 'paper_url', 's2_abstract',
       's2_body_text', 's2_paper_id', 's2_title', 'score', 'task', 'title',
       'dataset'],
      dtype='object')

In [128]:
from allennlp.data.dataset_readers.dataset_utils.span_utils import bioul_tags_to_spans

In [129]:
entity_mapping = {
    'Material' : 'dataset',
    'Method' : 'model_name',
    'Task' : 'task',
    'Metric' : 'metric'
}

In [130]:
def get_spans(taglist, wordlist) :
    entities = {}
    spans = bioul_tags_to_spans(taglist)
    for enttype, (start, end) in spans :
        if enttype not in entities :
            entities[enttype] = []
        entities[enttype].append(" ".join(wordlist[start:end+1]))
            
    return entities

In [139]:
def extract_entites(row) :
    document_entities = {}
    sentences = row['words']
    tags = row['tags']
    for wordlist, taglist in zip(sentences, tags) :
        entities = get_spans(taglist, wordlist)
        for enttype, entlist in entities.items() :
            if enttype not in document_entities :
                document_entities[enttype] = []
            document_entities[enttype] += entlist
    return pd.Series(document_entities)

In [145]:
pwc_sentences_entities = pwc_sentences.apply(extract_entites, axis=1)

In [149]:
pwc_sentences = pd.concat([pwc_sentences, pwc_sentences_entities], axis=1)

In [164]:
def get_aggregated_field(field) :
    df_agg = pwc_df.groupby(['s2_paper_id'])[field].aggregate(lambda x : tuple(set(list(x)))).reset_index()
    return df_agg

In [275]:
def compare_true_and_predicted_field(true_field, predicted_field) :
    predicted_df = pwc_sentences[['doc_id', 'text_field', predicted_field, 'sentence']]
#     assert predicted_df.doc_id.duplicated().sum() == 0
    true_df = get_aggregated_field(true_field)
    joined = true_df.merge(predicted_df, left_on=['s2_paper_id'], right_on=['doc_id'])
    
    joined['sentence'] = joined['sentence'].apply(lambda x : " ".join(x))
    joined[predicted_field] = joined[predicted_field].apply(lambda x : " ".join(x) if x == x else '')
    
    def compare_fields(row) :
        row['in_full'] = tuple([str(x) in row['sentence'] for x in row[true_field]])
        row['in_predicted'] = tuple([str(x) in row[predicted_field] for x in row[true_field]])
        row['n_total'] = len(row[true_field])
        row['n_found_full'] = sum(row['in_full'])
        row['n_found_predicted'] = sum(row['in_predicted'])
        return row
    
    joined = joined.apply(compare_fields, axis=1)

    return joined

In [276]:
def aggregate_across_text_field(oracle) :
    def collapse(rows) :
        in_full = [any(x) for x in zip(*list(rows['in_full']))]
        in_pred = [any(x) for x in zip(*list(rows['in_predicted']))]
        
        return pd.Series({'in_full' : sum(in_full), 'in_predicted' : sum(in_pred), 'total' : len(in_full)})
    oracle = oracle.groupby(['s2_paper_id', 'doc_id'])[['in_full', 'in_predicted']].apply(collapse)
    return oracle

In [268]:
oracle = compare_true_and_predicted_field('model_name', 'Method')
aggregated_dataset_values = aggregate_across_text_field(oracle)

In [277]:
predicted_entities = ['Material', 'Metric', 'Task', 'Method', 'Generic', 'OtherScientificTerm']
true_entities = ['metric', 'model_name', 'task', 'score', 'dataset']

In [278]:
results = []

for pe in tqdm(predicted_entities) :
    for te in tqdm(true_entities) :
        oracle = compare_true_and_predicted_field(te, pe)
        aggregated_dataset_values = aggregate_across_text_field(oracle).sum()
        aggregated_dataset_values['predicted'] = pe
        aggregated_dataset_values['true'] = te
        results.append(aggregated_dataset_values)

100%|██████████| 6/6 [05:15<00:00, 52.78s/it]


In [281]:
results = pd.DataFrame(results)

In [285]:
results['recall_oracle'] = results['in_full'] / results['total'] * 100
results['recall_pred'] = results['in_predicted'] / results['total'] * 100

In [286]:
results

,in_full,in_predicted,total,predicted,true,recall_oracle,recall_pred
0,558,44,2117,Material,metric,26.358054,2.078413
1,578,94,1607,Material,model_name,35.967642,5.849409
2,489,60,1406,Material,task,34.779516,4.267425
3,540,17,4915,Material,score,10.986775,0.345880
4,852,761,2921,Material,dataset,29.168093,26.052722
5,558,448,2117,Metric,metric,26.358054,21.162022
6,578,41,1607,Metric,model_name,35.967642,2.551338
7,489,1,1406,Metric,task,34.779516,0.071124
8,540,33,4915,Metric,score,10.986775,0.671414
9,852,69,2921,Metric,dataset,29.168093,2.362205
